In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from pathlib import Path

## Screened images

In [16]:
screened_files = list(set(['26_EIPH_566482 L Berliner Blau.svs',
                 '22_EIPH_575216 Berliner Blau.svs',
                 '25_EIPH_568150 Berliner Blau.svs',
                 '20_EIPH_576150 berliner blau.svs',
                 '15_EIPH_568320 berliner blau.svs',
                 '02_EIPH_574162 berliner blau.svs',
                 '18_EIPH_574882 R Berliner Blau.svs',
                 '13_EIPH_570370 Berliner Blau.svs',
                 '16_EIPH_576172 Turnbull blue-002.svs',
                 '19_EIPH_566933 L Berliner Blau.svs',
                 '04_EIPH_567017 Turnbull blue-001.svs',
                 '29_EIPH_566481 L Turnbull blue.svs',
                 '09_EIPH_563417 L berliner blau.svs',
                 '29_EIPH_566481 L Berliner Blau.svs',
                 '20_EIPH_576150 Turnbull blue.svs',
                 '27_EIPH_571557 berliner blau.svs',
                 '11_EIPH_575697 Turnbull blue-002.svs',
                 '31_EIPH_568354 Berliner Blau.svs',
                 '07_EIPH_574882 Berliner Blau-001.svs',

                 '14_EIPH_568381 berliner blau-001.svs',
                 '01_EIPH_563479 Berliner Blau.svs',
                 '08_EIPH_574999 Berliner Blau.svs',
                 '16_EIPH_574882 R Berliner Blau.svs',
                 '17_EIPH_575796 Berliner Blau.svs',
                 '11_EIPH_575697 Berliner Blau.svs',
                 '23_EIPH_563476 Turnbull blue.svs',
                 '01_EIPH_563479 Turnbull blue.svs',
                 '24_EIPH_576255 Turnbull blue.svs',
                 '02_EIPH_574162 Turnbull blue-001.svs',
                 '27_EIPH_571557 Turnbull blue.svs',
                 '10_EIPH_563417 R Berliner Blau.svs',
                 '25_EIPH_568150 Turnbull blue.svs',
                 '21_EIPH_566482 R Turnbull blue.svs',
                 '09_EIPH_563417 L Turnbull blue.svs',
                 '05_EIPH_569923 Berliner Blau.svs',
                 '05_EIPH_569923 Turnbull blue.svs',
                 '04_EIPH_567017 Berliner Blau.svs',

                 '14_EIPH_568381 Turnbull blue.svs',
                 '23_EIPH_563476 Berliner Blau-001.svs',
                 '24_EIPH_576255 Berliner Blau.svs',
                 '30_EIPH_588355 Berliner Blau.svs',
                 '18_EIPH_574882 Turnbull blue-002.svs',
                 '15_EIPH_568320 Turnbull blue.svs',
                 '17_EIPH_575796 Turnbull blue.svs',
                 '30_EIPH_568355 Turnbull blue.svs',
                 '22_EIPH_575216 Turnbull blue.svs',
                 '13_EIPH_570370 Turnbull blue-001.svs',
                 '31_EIPH_568354 Turnbull blue.svs',
                 '03_EIPH_566933 R Berliner Blau neu.svs',
                 '28_EIPH_569948 L berliner blau.svs',
                 '07_EIPH_574882 left lung Turnbull blue.svs',
                 '19_EIPH_566933 L Tunrbull blue.svs',
                 '28_EIPH_569948 L Turnbull blue.svs',
                 '10_EIPH_563417 R Turnbull blue-002.svs',
                 '03_EIPH_566933 R Turnbull blue.svs',
                 
                # Cat
                "BAL_Cat_Turnbull blue_13.svs", 
                "BAL_Cat_Turnbull blue_6.svs", 
                "BAL_Cat_Turnbull blue_1.svs", 
                "BAL_Cat_Turnbull blue_12.svs", 
                "BAL_Cat_Turnbull blue_14.svs", 
                "BAL_Cat_Turnbull blue_10.svs", 
                "BAL_Cat_Turnbull blue_2.svs",


                # Human
                "10835_19 humane BAL Turnbull Blau.svs",
                "10120_19 humane BAL Berliner Blau.svs",
                "10052_19 humane BAL Berliner Blau-001.svs",
                "10080_19 humane BAL Berliner Blau-001.svs",
                "2707_20 TB BAL Human neu.svs",
                "10835_19 humane BAL Berliner Blau.svs",
                "10227_19 humane BAL Berliner Blau.svs",
                "11480_19 humane BAL Berliner Blau.svs",
                "11323_19 humane BAL Berliner Blau.svs",
                "2702_20 BB Human BAL-001.svs",
                "10672_19 humane BAL Berliner Blau.svs",
                "10672_19 humane BAL Turnbull Blau.svs",]))

In [18]:
screened_tiff_files = list(set(['26_EIPH_566482 L Berliner Blau.tiff',
                 '22_EIPH_575216 Berliner Blau.tiff',
                 '25_EIPH_568150 Berliner Blau.tiff',
                 '20_EIPH_576150 berliner blau.tiff',
                 '15_EIPH_568320 berliner blau.tiff',
                 '02_EIPH_574162 berliner blau.tiff',
                 '18_EIPH_574882 R Berliner Blau.tiff',
                 '13_EIPH_570370 Berliner Blau.tiff',
                 '16_EIPH_576172 Turnbull blue-002.tiff',
                 '19_EIPH_566933 L Berliner Blau.tiff',
                 '04_EIPH_567017 Turnbull blue-001.tiff',
                 '29_EIPH_566481 L Turnbull blue.tiff',
                 '09_EIPH_563417 L berliner blau.tiff',
                 '29_EIPH_566481 L Berliner Blau.tiff',
                 '20_EIPH_576150 Turnbull blue.tiff',
                 '27_EIPH_571557 berliner blau.tiff',
                 '11_EIPH_575697 Turnbull blue-002.tiff',
                 '31_EIPH_568354 Berliner Blau.tiff',
                 '07_EIPH_574882 Berliner Blau-001.tiff',

                 '14_EIPH_568381 berliner blau-001.tiff',
                 '01_EIPH_563479 Berliner Blau.tiff',
                 '08_EIPH_574999 Berliner Blau.tiff',
                 '16_EIPH_574882 R Berliner Blau.tiff',
                 '17_EIPH_575796 Berliner Blau.tiff',
                 '11_EIPH_575697 Berliner Blau.tiff',
                 '23_EIPH_563476 Turnbull blue.tiff',
                 '01_EIPH_563479 Turnbull blue.tiff',
                 '24_EIPH_576255 Turnbull blue.tiff',
                 '02_EIPH_574162 Turnbull blue-001.tiff',
                 '27_EIPH_571557 Turnbull blue.tiff',
                 '10_EIPH_563417 R Berliner Blau.tiff',
                 '25_EIPH_568150 Turnbull blue.tiff',
                 '21_EIPH_566482 R Turnbull blue.tiff',
                 '09_EIPH_563417 L Turnbull blue.tiff',
                 '05_EIPH_569923 Berliner Blau.tiff',
                 '05_EIPH_569923 Turnbull blue.tiff',
                 '04_EIPH_567017 Berliner Blau.tiff',

                 '14_EIPH_568381 Turnbull blue.tiff',
                 '23_EIPH_563476 Berliner Blau-001.tiff',
                 '24_EIPH_576255 Berliner Blau.tiff',
                 '30_EIPH_588355 Berliner Blau.tiff',
                 '18_EIPH_574882 Turnbull blue-002.tiff',
                 '15_EIPH_568320 Turnbull blue.tiff',
                 '17_EIPH_575796 Turnbull blue.tiff',
                 '30_EIPH_568355 Turnbull blue.tiff',
                 '22_EIPH_575216 Turnbull blue.tiff',
                 '13_EIPH_570370 Turnbull blue-001.tiff',
                 '31_EIPH_568354 Turnbull blue.tiff',
                 '03_EIPH_566933 R Berliner Blau neu.tiff',
                 '28_EIPH_569948 L berliner blau.tiff',
                 '07_EIPH_574882 left lung Turnbull blue.tiff',
                 '19_EIPH_566933 L Tunrbull blue.tiff',
                 '28_EIPH_569948 L Turnbull blue.tiff',
                 '10_EIPH_563417 R Turnbull blue-002.tiff',
                 '03_EIPH_566933 R Turnbull blue.tiff',
                 
                # Cat
                "BAL_Cat_Turnbull blue_13.tiff", 
                "BAL_Cat_Turnbull blue_6.tiff", 
                "BAL_Cat_Turnbull blue_1.tiff", 
                "BAL_Cat_Turnbull blue_12.tiff", 
                "BAL_Cat_Turnbull blue_14.tiff", 
                "BAL_Cat_Turnbull blue_10.tiff", 
                "BAL_Cat_Turnbull blue_2.tiff",


                # Human
                "10835_19 humane BAL Turnbull Blau.tiff",
                "10120_19 humane BAL Berliner Blau.tiff",
                "10052_19 humane BAL Berliner Blau-001.tiff",
                "10080_19 humane BAL Berliner Blau-001.tiff",
                "2707_20 TB BAL Human neu.tiff",
                "10835_19 humane BAL Berliner Blau.tiff",
                "10227_19 humane BAL Berliner Blau.tiff",
                "11480_19 humane BAL Berliner Blau.tiff",
                "11323_19 humane BAL Berliner Blau.tiff",
                "2702_20 BB Human BAL-001.tiff",
                "10672_19 humane BAL Berliner Blau.tiff",
                "10672_19 humane BAL Turnbull Blau.tiff",]))

# Load annotation versions

In [3]:
def set_additional_fields(frame):
    
    frame["width"] = [vector["x2"] - vector["x1"] for vector in frame['vector']]
    frame["height"] = [vector["y2"] - vector["y1"] for vector in frame['vector']]
    frame["scales"] = frame["width"] / frame["height"]
    
    return frame

In [25]:
#species = "Cat", "Equine", "Human"

v0 = pd.concat([
    pd.read_pickle(f"../EIPH_Cat_SDATA_Inference_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Human_SDATA_Inference_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Equine_SDATA_Inference_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Equine_SREP_Annotations.pkl")
])
v0 = v0[v0["image_name"].isin(screened_files)]
v0 = set_additional_fields(v0)
print(f"Inference: {len(v0)}")

v_cluster = pd.concat([
    pd.read_pickle(f"../EIPH_Cat_SDATA_Cluster_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Human_SDATA_Cluster_Annotations.pkl"),
])
v_cluster = v_cluster[v_cluster["image_name"].isin(screened_files)]
v_cluster = set_additional_fields(v_cluster)
print(f"Cluster: {len(v_cluster)}")

v_screening = pd.concat([
    pd.read_pickle(f"../EIPH_Cat_SDATA_Screening_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Human_SDATA_Screening_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Equine_SDATA_Screening_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_SREP_Screening_Annotations.pkl"),
])
v_screening = v_screening[v_screening["image_name"].isin(screened_files)]
v_screening = set_additional_fields(v_screening)
print(f"Screening: {len(v_screening)}")

v_density = pd.concat([
    pd.read_pickle(f"../EIPH_Cat_SDATA_Density_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Human_SDATA_Density_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_Equine_SDATA_Density_Annotations.pkl"),
    pd.read_pickle(f"../EIPH_SREP_Density_Annotations.pkl"),
])
v_density = v_density[v_density["image_name"].isin(screened_tiff_files)]
print(f"Density: {len(v_density)}")

Inference: 585600
Cluster: 128012
Screening: 303289
Density: 297383


In [5]:
v0.head()

,id,image_id,image_set,species,image_name,image_type,grade,vector,unique_identifier,user_id,deleted,last_editor,data_set_name,version,width,height,scales
0,1049539,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 9903.0, 'x2': 10029.0, 'y1': 5508.0, 'y...",875aab61-3f1f-429a-a053-f87aba6c99da,1,False,1,SDATA,Inference,126.0,129.0,0.976744
1,1049540,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 26435, 'x2': 26544, 'y1': 24003, 'y2': ...",37e8ad72-ba58-4608-aeab-cfdf004d1f00,1,True,12,SDATA,Inference,109.0,109.0,1.000000
2,1049542,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 18462, 'x2': 18588, 'y1': 10556, 'y2': ...",37f441f4-39ec-478c-9718-c04c4a11c914,1,True,12,SDATA,Inference,126.0,126.0,1.000000
3,1049545,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 23213.0, 'x2': 23359.0, 'y1': 5401.0, '...",74862e27-5881-41d8-90f3-5eb305d35e41,1,True,1,SDATA,Inference,146.0,104.0,1.403846
4,1049548,3512,222,Cat,BAL_Cat_Turnbull blue_1.svs,Turnbull,0,"{'x1': 12243.0, 'x2': 12375.0, 'y1': 14740.0, ...",d718f087-7a46-4e22-8610-ba5f82d04818,1,False,1,SDATA,Inference,132.0,132.0,1.000000


## Global Stats

In [26]:
def get_global_stats(frame, dataset , version, species):
    
    nr_slides = len(frame["image_name"].unique())
    total = frame.shape[0]
    score = int(frame["grade"].mean() * 100)
    _0 = frame[frame["grade"] == 0].shape[0]
    _1 = frame[frame["grade"] == 1].shape[0]
    _2 = frame[frame["grade"] == 2].shape[0]
    _3 = frame[frame["grade"] == 3].shape[0]
    _4 = frame[frame["grade"] == 4].shape[0]
    mean = frame["grade"].mean()
    SD = frame["grade"].std()
    
    frame = [[species, dataset, nr_slides, version, total, score, _0, _1, _2, _3, _4, mean, SD]]
    
    return pd.DataFrame(frame, columns=["species", "dataset" ,"slides", "version", "total", "score", "0", "1", "2", "3", "4", "mean", "SD"])

In [27]:
v0_meta = get_global_stats(v0, "Total", "Inference", "Total")
v_cluster_meta = get_global_stats(v_cluster, "Total", "Cluster", "Total")
v_screening_meta = get_global_stats(v_screening, "Total", "Screening", "Total")
v_density_meta = get_global_stats(v_density, "Total", "Density", "Total")

metaDf = pd.concat([v0_meta, v_cluster_meta, v_screening_meta, v_density_meta])
metaDf.head()

,species,dataset,slides,version,total,score,0,1,2,3,4,mean,SD
0,Total,Total,74,Inference,585600,98,216835,208017,119409,36798,4541,0.982570,0.946059
0,Total,Total,12,Cluster,128012,133,21532,53704,42553,8932,1291,1.334016,0.871160
0,Total,Total,74,Screening,303289,113,92522,109169,74122,24459,3017,1.130466,0.971879
0,Total,Total,74,Density,297383,109,86551,116789,74766,17795,1482,1.094999,0.904234


## Stain 

In [10]:
"BerlinerBlaue: {}".format(len([name for name in v0["image_name"].unique() if "erliner" in name]))

'BerlinerBlaue: 0'

In [11]:
"TurnbullBlaue: {}".format(len([name for name in v0["image_name"].unique() if "erliner" not in name]))

'TurnbullBlaue: 7'

## Introduced new cells?

In [12]:
188

188

## Images

In [13]:
v0["image_name"].unique()

array(['BAL_Cat_Turnbull blue_1.svs', 'BAL_Cat_Turnbull blue_2.svs',
       'BAL_Cat_Turnbull blue_6.svs', 'BAL_Cat_Turnbull blue_10.svs',
       'BAL_Cat_Turnbull blue_12.svs', 'BAL_Cat_Turnbull blue_13.svs',
       'BAL_Cat_Turnbull blue_14.svs'], dtype=object)

In [14]:
len(v0["image_name"].unique())

7